# Environment Setup
## Installing required dependencies
This notebook uses several libraries and dependencies. Run the below commands to install the required libraries and dependencies before continuing:

In [8]:
# Dependency installation
# e.g. pip install pandas

In [9]:
#Dependecy installation
import ipywidgets as widgets
import os
from IPython.display import Javascript, display, clear_output

In [12]:
#get all folder in outputs dir
list = os.listdir("outputs")

config = {'output_dir' : ''}
# create dropdown list
dropdown_list = widgets.Dropdown(options=list,description='Folder Name:',disabled=False)

def on_change(change):
     config['output_dir'] = change.new
    
dropdown_list.observe(on_change, names='value')
display(dropdown_list)


textfield = widgets.Text(value='', disabled=False, description= 'OUTPUT_DIR:')

#Create output widgets to display popup alert
notify_output = widgets.Output()
display(notify_output)

@notify_output.capture()
def popup(text, created):
    clear_output()
    if created:
        text = f"Folder {text} created successfully"
    else:
        text =f"Folder {text} not created successfully"
        
    display(Javascript("alert('{}')".format(text)))
        
    
#function to change output_dir location when button on clicked
def on_button_clicked(b):
    folder_name = textfield.value
    if not os.path.exists("outputs/"+ folder_name):
      os.makedirs("outputs/"+ folder_name)
      dropdown_list.options = os.listdir("outputs")
      popup(folder_name,True)
    else:
      popup(folder_name,False)
        
        
#create textfield widgets
label = widget.Label(value = "Create new Folder:")
textfield = widgets.Text(value='', disabled=False, description= '')

#create submit button
button = widgets.Button(description="Submit")
button.on_click(on_button_clicked)
widgets.HBox([label,textfield,button])


Dropdown(description='Folder Name:', options=('2', 'default'), value='2')

Output()

In [11]:
config

{'output_dir': ''}